In [30]:
! wget --no-check-certificate "https://t1.ipcamlive.com/timelapses/5aec8d9905b9a/2023-07-17/video.mp4"

--2023-09-18 12:37:01--  https://t1.ipcamlive.com/timelapses/5aec8d9905b9a/2023-07-17/video.mp4
Resolving t1.ipcamlive.com (t1.ipcamlive.com)... 173.198.196.110
Connecting to t1.ipcamlive.com (t1.ipcamlive.com)|173.198.196.110|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 35243285 (34M) [video/mp4]
Saving to: ‘video.mp4’

video.mp4           100%[===================>]  33,61M  8,94MB/s    in 4,3s    

2023-09-18 12:37:06 (7,78 MB/s) - ‘video.mp4’ saved [35243285/35243285]



Frame Extraction from the TimeLapse Video (Ocean City Inlet Parking Lot)

In [1]:
import cv2
import datetime

# Function to extract the frame closest to the target time
def extract_frame_by_time(video_file, target_time, output_file, video_start_time, video_end_time):
    cap = cv2.VideoCapture(video_file)

    if not cap.isOpened():
        print("Error: Could not open video file.")
        return

    # Convert the video start time to a datetime object
    video_start_time = datetime.datetime.strptime(video_start_time, "%H:%M")
    video_end_time = datetime.datetime.strptime(video_end_time, "%H:%M")

    # Calculate the target time with the offset time
    target_datetime = datetime.datetime.strptime(target_time, "%H:%M")

    # Calculate the total number of frames in the video
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

    # Calculate the video duration in seconds
    video_duration_seconds = (video_end_time - video_start_time).total_seconds()

    # Calculate the frame rate (frames per second)
    frame_rate = total_frames / video_duration_seconds

    # Calculate the target frame number based on the actual start time
    target_frame_number = int((target_datetime - video_start_time).total_seconds() * frame_rate)

    # Check if the target frame number is within the bounds of the video
    if target_frame_number < 0 or target_frame_number >= total_frames:
        print("No matching frame found within the video.")
        return

    cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_number)

    ret, frame = cap.read()

    if ret:
        cv2.imwrite(output_file, frame)
        print(f"Frame extracted at {target_time}.")
    else:
        print("No matching frame found within the video.")

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_file = "video_07_17.mp4"  # Replace with your video file path
    target_time = "20:36"  # Replace with the desired target time in 24-hour format
    output_file = "output_frame1.jpg"  # Replace with the desired output file path
    video_start_time = "00:10"  # Replace with the video start time in 24-hour format
    video_end_time = "22:23"  # Replace with the video end time in 24-hour format

    extract_frame_by_time(video_file, target_time, output_file, video_start_time, video_end_time)



Frame extracted at 20:36.


In [ ]:
# import cv2
# import datetime

# # Function to extract the frame closest to the target time
# def extract_frame_by_time(video_file, target_time, output_file, video_start_time):
#     cap = cv2.VideoCapture(video_file)

#     if not cap.isOpened():
#         print("Error: Could not open video file.")
#         return

#     # Convert the video start time to a datetime object
#     video_start_time = datetime.datetime.strptime(video_start_time, "%H:%M")

#     # Calculate the target time with the offset time
#     target_datetime = datetime.datetime.strptime(target_time, "%H:%M")
#     target_datetime = target_datetime + datetime.timedelta(hours=video_start_time.hour, minutes=video_start_time.minute)

#     # Calculate the total number of frames in the video
#     total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

#     # Calculate the target frame number
#     target_frame_number = (target_datetime - video_start_time).total_seconds() * total_frames / (24 * 3600)

#     cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_number)

#     ret, frame = cap.read()

#     if ret:
#         cv2.imwrite(output_file, frame)
#         print(f"Frame extracted at {target_time}.")
#     else:
#         print("No matching frame found within the video.")

#     cap.release()
#     cv2.destroyAllWindows()

# if __name__ == "__main__":
#     video_file = "video_08_27.mp4"  # Replace with your video file path
#     target_time = "13:55"  # Replace with the desired target time in 24-hour format
#     output_file = "output_frame.jpg"  # Replace with the desired output file path
#     video_start_time = "00:11"  # Replace with the video start time in 24-hour format

#     extract_frame_by_time(video_file, target_time, output_file, video_start_time)
